In [3]:
import pandas as pd
from os import listdir
import numpy as np
from os.path import isfile, join
from Bio import SeqIO
import re
HOME_DIR = "../data/pdb_str/"
HITS_DIR = "../data/pdb_str/pb_hits/"
PDB_DIR = "../data/pdb_str/pb_aa/"
ALPHA_DIR = "../data/pdb_str/pb/"

In [11]:
clearEmpty = lambda x: [ i for i in x if i != '']
regex = re.compile('[^0-9]')

def parse_interval(range):
    segs = [ seg.split("-")  for seg in range.split(",")]
    segs = [ clearEmpty(s) for s in segs ]
    return [ [ int(regex.sub('', s[0])), int(regex.sub('', s[1]).split(":")[0]) ] for s in segs]

def merge_range(elem):
    return ",".join([ ":".join( [str(o) for o in pair] ) for pair in elem])

def readPDB(path):
    for record in SeqIO.parse(path, "fasta"):
        pdb = "".join(list(record.seq))
    return pdb

def scale_hit(pdb_seq, start, end):
    hit_interval = []
    num_char_hit = (end - start + 1)
    num_char_viewed_hit = 0
    num_char_viewed_seq = 0

    for I in pdb_seq:
        start_scaled = max((I[0] + start - 1) - num_char_viewed_seq, I[0])
        end_scaled = min(start_scaled+num_char_hit-1, I[1])
        if start_scaled <= I[1] and num_char_hit > 0:
            hit_interval = hit_interval + [[start_scaled, end_scaled]]
            num_char_hit = num_char_hit - (end_scaled-start_scaled+1)
            num_char_viewed_hit = end_scaled - start_scaled + 1
            num_char_viewed_seq = num_char_viewed_seq + (start_scaled-start)+num_char_viewed_hit
        else:
            num_char_viewed_seq = num_char_viewed_seq +(I[1]-I[0])+1

    return hit_interval

In [3]:
files = [ "".join([HITS_DIR, f]) for f in listdir(HITS_DIR) ]
files = files[1:-1]
files[0:4]

['../data/pdb_str/pb_hits/hits_xaa_cath_pairs_aa.gz',
 '../data/pdb_str/pb_hits/hits_xaa_cath_pairs_ab.gz',
 '../data/pdb_str/pb_hits/hits_xaa_cath_pairs_ac.gz',
 '../data/pdb_str/pb_hits/hits_xaa_cath_pairs_ad.gz']

In [4]:
pdb_intervals = pd.read_csv("data/cath-b-newest-all", sep=" ", names =["pdb", "version", "cath", "range"])
pdb_intervals = pdb_intervals[["pdb", "range"]]
pdb_intervals.head()

,pdb,range
0,101mA00,0-153:A
1,102lA00,1-162:A
2,102mA00,0-153:A
3,103lA00,1-162:A
4,103mA00,0-153:A


In [86]:
def f10(f):
    _df = pd.read_csv(f)
    return _df[_df.hit_length >= 10]

dfs = [ f10(f) for f in files ]
df = pd.concat(dfs)

In [87]:
df.to_csv("../data/pdb_str/pb_hits/all_hits_f10.csv", index=False)

In [2]:
df = pd.read_csv("../data/pdb_str/pb_hits/all_hits_f10.csv")
df.shape

(31051346, 4)

In [3]:
df = df[df.hit.apply(lambda x: len(set(x)) >= 10) & (df.hit_length >= 20)]
df.shape

(654790, 4)

In [6]:
df["x_chain"] = df.x.apply(lambda x: x[0:-3])
df["y_chain"] = df.y.apply(lambda x: x[0:-3])
df = df.loc[df[["x_chain"]].drop_duplicates().index]
df = df.loc[df[["y_chain"]].drop_duplicates().index]
df.shape

(626, 6)

In [2]:
aa_filter = pd.read_csv("hit_aa_sample.csv")
# aa_filter = aa_filter[aa_filter.hit_length >= 10]
aa_filter["key"] = aa_filter[["x", "y"]].apply(lambda x: "%s_%s" % (x.x, x.y), axis=1)
aa_filter["key_revert"] = aa_filter[["x", "y"]].apply(lambda x: "%s_%s" % (x.y, x.x), axis=1)
aa_filter.shape

NameError: name 'pd' is not defined

In [9]:
df["key"] = df[["x", "y"]].apply(lambda x: "%s_%s" % (x.x, x.y), axis=1)
df = pd.merge(df, aa_filter[["key", "key_revert"]], on="key", how="left")
df = pd.merge(df, aa_filter[["key", "key_revert"]], left_on="key", right_on="key_revert", how="left")
df = df[df.key_revert_x.isna() & df.key_revert_y.isna()]
df.shape

(624, 10)

In [12]:
df.loc[:,"x_pdb_seq"] = df["x"].apply(lambda x: readPDB("%s%s.fasta" % (PDB_DIR, x[0:-2])))
df.loc[:,"x_pb_seq"] = df["x"].apply(lambda x: readPDB("%s%s.fasta" % (ALPHA_DIR, x[0:-2])))
df.loc[:,"y_pdb_seq"] = df["y"].apply(lambda x: readPDB("%s%s.fasta" % (PDB_DIR, x[0:-2])))
df.loc[:,"y_pb_seq"] = df["y"].apply(lambda x: readPDB("%s%s.fasta" % (ALPHA_DIR, x[0:-2])))
df.head()

,x,y,hit_length,hit,x_chain,y_chain,key_x,key_revert_x,key_y,key_revert_y,x_pdb_seq,x_pb_seq,y_pdb_seq,y_pb_seq
0,12asA00,1a99A01,21,DDDDDDEHIACFKLMMMMMMM,12as,1a99,12asA00_1a99A01,NaN,NaN,NaN,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...,ZZLMMMMMMMMMMMMMMMMMMMNOPACDDDDFBDCEHILPACEKJO...,QKTLHIYNWSDYIAPDTVANFEKETGIKVVYDVFDSNEVLEGKLMA...,ZZDDDDDDEHIACFKLMMMMMMMNOPACDDDDDFBFKLMMMMMMNO...
1,132lA00,4oyvA01,21,AFKLMMMMMMMMNOOPAKBCC,132l,4oyv,132lA00_4oyvA01,NaN,NaN,NaN,XVFGRCELAAAMXRHGLDNYRGYSLGNWVCAAXFESNFNTQATNRN...,ZZDFKLMMMMMMMNOPGOPGHIAFKLMMMMMMMMNOOPAKBCCDDF...,PTALERVQKEGVLRVITRNSPATYFQDRNGETGFEYELAKRFAERL...,ZZKLMMMMNOPACDDDDDDFKLOCCDFKOPEDFKLMMMMMMMMMNO...
2,155cA00,1q1lB00,21,HIAGHIACDFKLMMMMMMMMM,155c,1q1l,155cA00_1q1lB00,NaN,NaN,NaN,XNEGDAAKGEKEFNKCKACHMIQAPDGTDIKGGKTGPNLYGVVGRK...,ZZXPFKMMMMMMMXXXXXXXXXDCKBCEDJDDFKBACFEHJAOIAG...,SLRYLRFLTAGESHGKGLTAILEGIPANLPLSEEEINHELRRRQRG...,ZZMBGCDDEDDFKLNBDCDDDEHIAGHIACDFKLMMMMMMMMMMMX...
3,16pkA02,1a40A02,28,MMMMMMNOPACDDFKLMMMMMMMMMMMM,16pk,1a40,16pkA02_1a40A02,NaN,NaN,NaN,EKKSINECDLKGKKVLIRVDFNVPVKNGKITNDYRIRSALPTLKKV...,ZZCFKLPCFEHPACDDDDDFBBCDEEHIAFBFKLMMMMMMMMMMMM...,EASLTGAGATFPAPVYAKWADTYQKETGNKVNYQGIGSSGGVKQII...,ZZDDDEDJFKLMMMMMMMMMMMMMMNOPACDDDDEBJKLMMMMMMN...
4,16vpA00,1a1vA01,21,FKLMMMMMMMMMMNOPACDDD,16vp,1a1v,16vpA00_1a1vA01,NaN,NaN,NaN,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...,ZZDCDDDDDDFKLMMMMMMMMNOPAKMMMMMMMMMMPMBDFKLMPC...,PPAVPQSFQVAHLHAPTGSGKSTKVPAAYAAQGYKVLVLNPSVAAT...,ZZDDFBDCDDDDDDDEHILMMMMMMMMMMMNOPACDDDDFBLKLMM...


In [14]:
df.loc[:, "x_hit_start"] = df[["x_pb_seq", "hit"]].apply(lambda x: x.x_pb_seq.find(x.hit), axis=1)
df.loc[:, "x_hit_end"]   = df[["x_hit_start", "hit"]].apply(lambda x: x.x_hit_start + len(x.hit), axis=1)
df.loc[:, "y_hit_start"] = df[["y_pb_seq", "hit"]].apply(lambda x: x.y_pb_seq.find(x.hit), axis=1)
df.loc[:, "y_hit_end"]   = df[["y_hit_start", "hit"]].apply(lambda x: x.y_hit_start + len(x.hit), axis=1)
df.head()

,x,y,hit_length,hit,x_chain,y_chain,key_x,key_revert_x,key_y,key_revert_y,x_pdb_seq,x_pb_seq,y_pdb_seq,y_pb_seq,x_hit_start,x_hit_end,y_hit_start,y_hit_end
0,12asA00,1a99A01,21,DDDDDDEHIACFKLMMMMMMM,12as,1a99,12asA00_1a99A01,NaN,NaN,NaN,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...,ZZLMMMMMMMMMMMMMMMMMMMNOPACDDDDFBDCEHILPACEKJO...,QKTLHIYNWSDYIAPDTVANFEKETGIKVVYDVFDSNEVLEGKLMA...,ZZDDDDDDEHIACFKLMMMMMMMNOPACDDDDDFBFKLMMMMMMNO...,114,135,2,23
1,132lA00,4oyvA01,21,AFKLMMMMMMMMNOOPAKBCC,132l,4oyv,132lA00_4oyvA01,NaN,NaN,NaN,XVFGRCELAAAMXRHGLDNYRGYSLGNWVCAAXFESNFNTQATNRN...,ZZDFKLMMMMMMMNOPGOPGHIAFKLMMMMMMMMNOOPAKBCCDDF...,PTALERVQKEGVLRVITRNSPATYFQDRNGETGFEYELAKRFAERL...,ZZKLMMMMNOPACDDDDDDFKLOCCDFKOPEDFKLMMMMMMMMMNO...,22,43,271,292
2,155cA00,1q1lB00,21,HIAGHIACDFKLMMMMMMMMM,155c,1q1l,155cA00_1q1lB00,NaN,NaN,NaN,XNEGDAAKGEKEFNKCKACHMIQAPDGTDIKGGKTGPNLYGVVGRK...,ZZXPFKMMMMMMMXXXXXXXXXDCKBCEDJDDFKBACFEHJAOIAG...,SLRYLRFLTAGESHGKGLTAILEGIPANLPLSEEEINHELRRRQRG...,ZZMBGCDDEDDFKLNBDCDDDEHIAGHIACDFKLMMMMMMMMMMMX...,46,67,22,43
3,16pkA02,1a40A02,28,MMMMMMNOPACDDFKLMMMMMMMMMMMM,16pk,1a40,16pkA02_1a40A02,NaN,NaN,NaN,EKKSINECDLKGKKVLIRVDFNVPVKNGKITNDYRIRSALPTLKKV...,ZZCFKLPCFEHPACDDDDDFBBCDEEHIAFBFKLMMMMMMMMMMMM...,EASLTGAGATFPAPVYAKWADTYQKETGNKVNYQGIGSSGGVKQII...,ZZDDDEDJFKLMMMMMMMMMMMMMMNOPACDDDDEBJKLMMMMMMN...,172,200,283,311
4,16vpA00,1a1vA01,21,FKLMMMMMMMMMMNOPACDDD,16vp,1a1v,16vpA00_1a1vA01,NaN,NaN,NaN,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...,ZZDCDDDDDDFKLMMMMMMMMNOPAKMMMMMMMMMMPMBDFKLMPC...,PPAVPQSFQVAHLHAPTGSGKSTKVPAAYAAQGYKVLVLNPSVAAT...,ZZDDFBDCDDDDDDDEHILMMMMMMMMMMMNOPACDDDDFBLKLMM...,241,262,180,201


In [15]:
df.loc[:, "x_pdb_fragment"]  = df[["x_pdb_seq", "x_hit_start", "x_hit_end"]].apply(lambda x: x.x_pdb_seq[x.x_hit_start:x.x_hit_end], axis=1)
df.loc[:, "y_pdb_fragment"]  = df[["y_pdb_seq", "y_hit_start", "y_hit_end"]].apply(lambda x: x.y_pdb_seq[x.y_hit_start:x.y_hit_end], axis=1)

In [16]:
df.head()

,x,y,hit_length,hit,x_chain,y_chain,key_x,key_revert_x,key_y,key_revert_y,x_pdb_seq,x_pb_seq,y_pdb_seq,y_pb_seq,x_hit_start,x_hit_end,y_hit_start,y_hit_end,x_pdb_fragment,y_pdb_fragment
0,12asA00,1a99A01,21,DDDDDDEHIACFKLMMMMMMM,12as,1a99,12asA00_1a99A01,NaN,NaN,NaN,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...,ZZLMMMMMMMMMMMMMMMMMMMNOPACDDDDFBDCEHILPACEKJO...,QKTLHIYNWSDYIAPDTVANFEKETGIKVVYDVFDSNEVLEGKLMA...,ZZDDDDDDEHIACFKLMMMMMMMNOPACDDDDDFBFKLMMMMMMNO...,114,135,2,23,DWERVMGDGERQFSTLKSTVE,TLHIYNWSDYIAPDTVANFEK
1,132lA00,4oyvA01,21,AFKLMMMMMMMMNOOPAKBCC,132l,4oyv,132lA00_4oyvA01,NaN,NaN,NaN,XVFGRCELAAAMXRHGLDNYRGYSLGNWVCAAXFESNFNTQATNRN...,ZZDFKLMMMMMMMNOPGOPGHIAFKLMMMMMMMMNOOPAKBCCDDF...,PTALERVQKEGVLRVITRNSPATYFQDRNGETGFEYELAKRFAERL...,ZZKLMMMMNOPACDDDDDDFKLOCCDFKOPEDFKLMMMMMMMMMNO...,22,43,271,292,YSLGNWVCAAXFESNFNTQAT,TDWRLLAAIGYQESLWQPGAT
2,155cA00,1q1lB00,21,HIAGHIACDFKLMMMMMMMMM,155c,1q1l,155cA00_1q1lB00,NaN,NaN,NaN,XNEGDAAKGEKEFNKCKACHMIQAPDGTDIKGGKTGPNLYGVVGRK...,ZZXPFKMMMMMMMXXXXXXXXXDCKBCEDJDDFKBACFEHJAOIAG...,SLRYLRFLTAGESHGKGLTAILEGIPANLPLSEEEINHELRRRQRG...,ZZMBGCDDEDDFKLNBDCDDDEHIAGHIACDFKLMMMMMMMMMMMX...,46,67,22,43,IASEEGFKYGEGILEVAEKNP,EGIPANLPLSEEEINHELRRR
3,16pkA02,1a40A02,28,MMMMMMNOPACDDFKLMMMMMMMMMMMM,16pk,1a40,16pkA02_1a40A02,NaN,NaN,NaN,EKKSINECDLKGKKVLIRVDFNVPVKNGKITNDYRIRSALPTLKKV...,ZZCFKLPCFEHPACDDDDDFBBCDEEHIAFBFKLMMMMMMMMMMMM...,EASLTGAGATFPAPVYAKWADTYQKETGNKVNYQGIGSSGGVKQII...,ZZDDDEDJFKLMMMMMMMMMMMMMMNOPACDDDDEBJKLMMMMMMN...,172,200,283,311,MTGIPKILGNGAAGYLMEKEISYFAKVL,GAKQANDLDYASLPDSVVEQVRAAWKTN
4,16vpA00,1a1vA01,21,FKLMMMMMMMMMMNOPACDDD,16vp,1a1v,16vpA00_1a1vA01,NaN,NaN,NaN,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...,ZZDCDDDDDDFKLMMMMMMMMNOPAKMMMMMMMMMMPMBDFKLMPC...,PPAVPQSFQVAHLHAPTGSGKSTKVPAAYAAQGYKVLVLNPSVAAT...,ZZDDFBDCDDDDDDDEHILMMMMMMMMMMMNOPACDDDDFBLKLMM...,241,262,180,201,EARRLRELNHIREHLNLPLVR,SKKKCDELAAKLVALGINAVA


In [18]:
df.rename(columns={"hit" : "pb_fragment"})[["x", "y", "hit_length", "pb_fragment", "x_pdb_fragment", "y_pdb_fragment"]]\
.to_csv("pb_hits_sample.csv", index=False)

In [95]:
df[df.x.str.startswith("1vr5") & df.y.str.startswith("2o7i")]

,x,y,hit_length,hit,x_pdb_seq,x_pb_seq,y_pdb_seq,y_pb_seq,x_hit_start,x_hit_end,y_hit_start,y_hit_end,x_pdb_fragment,y_pdb_fragment
15583196,1vr5A01,2o7iA02,55,DDDDDDDEHILMLMMMMMMMMMMMMMNOPACDDDDDDFKLMMMMMM...,HERNKTLYWGGALWSPPSNWNPFTPWNAVAGTIGLVYEPLFLYDPL...,ZZKLCCCDEHJACBDCFBFCFKLMMLGCEHJAOJLPCCFBDCDFKN...,SLPREDTVYIGGALWGPATTWNLYAPQSTWGTDQFMYLPAFQYDLG...,ZZFKLCCCDEHKLGCJDFBDCFKBMBDCEHJLMMMMCCFBDCDFKL...,355,410,369,424,FKLTIECPYGWTDWMVSIQSIAEDLVKVGINVEPKYPDYSKYADDL...,GPYTISVPYGWTDWMMMCEMIAKNLRSIGIDVKTEFPDFSVWADRM...
15583197,1vr5A01,2o7iA03,55,DDDDDDDEHILMLMMMMMMMMMMMMMNOPACDDDDDDFKLMMMMMM...,HERNKTLYWGGALWSPPSNWNPFTPWNAVAGTIGLVYEPLFLYDPL...,ZZKLCCCDEHJACBDCFBFCFKLMMLGCEHJAOJLPCCFBDCDFKN...,SLPREDTVYIGGALWGPATTWNLYAPQSTWGTDQFMYLPAFQYDLG...,ZZFKLCCCDEHKLGCJDFBDCFKBMBDCEHJLMMMMCCFBDCDFKL...,355,410,369,424,FKLTIECPYGWTDWMVSIQSIAEDLVKVGINVEPKYPDYSKYADDL...,GPYTISVPYGWTDWMMMCEMIAKNLRSIGIDVKTEFPDFSVWADRM...
15605311,1vr5A02,2o7iA01,55,DDDDDDDEHILMLMMMMMMMMMMMMMNOPACDDDDDDFKLMMMMMM...,HERNKTLYWGGALWSPPSNWNPFTPWNAVAGTIGLVYEPLFLYDPL...,ZZKLCCCDEHJACBDCFBFCFKLMMLGCEHJAOJLPCCFBDCDFKN...,SLPREDTVYIGGALWGPATTWNLYAPQSTWGTDQFMYLPAFQYDLG...,ZZFKLCCCDEHKLGCJDFBDCFKBMBDCEHJLMMMMCCFBDCDFKL...,355,410,369,424,FKLTIECPYGWTDWMVSIQSIAEDLVKVGINVEPKYPDYSKYADDL...,GPYTISVPYGWTDWMMMCEMIAKNLRSIGIDVKTEFPDFSVWADRM...
15605312,1vr5A02,2o7iA03,55,DDDDDDDEHILMLMMMMMMMMMMMMMNOPACDDDDDDFKLMMMMMM...,HERNKTLYWGGALWSPPSNWNPFTPWNAVAGTIGLVYEPLFLYDPL...,ZZKLCCCDEHJACBDCFBFCFKLMMLGCEHJAOJLPCCFBDCDFKN...,SLPREDTVYIGGALWGPATTWNLYAPQSTWGTDQFMYLPAFQYDLG...,ZZFKLCCCDEHKLGCJDFBDCFKBMBDCEHJLMMMMCCFBDCDFKL...,355,410,369,424,FKLTIECPYGWTDWMVSIQSIAEDLVKVGINVEPKYPDYSKYADDL...,GPYTISVPYGWTDWMMMCEMIAKNLRSIGIDVKTEFPDFSVWADRM...
20709492,1vr5A03,2o7iA01,55,DDDDDDDEHILMLMMMMMMMMMMMMMNOPACDDDDDDFKLMMMMMM...,HERNKTLYWGGALWSPPSNWNPFTPWNAVAGTIGLVYEPLFLYDPL...,ZZKLCCCDEHJACBDCFBFCFKLMMLGCEHJAOJLPCCFBDCDFKN...,SLPREDTVYIGGALWGPATTWNLYAPQSTWGTDQFMYLPAFQYDLG...,ZZFKLCCCDEHKLGCJDFBDCFKBMBDCEHJLMMMMCCFBDCDFKL...,355,410,369,424,FKLTIECPYGWTDWMVSIQSIAEDLVKVGINVEPKYPDYSKYADDL...,GPYTISVPYGWTDWMMMCEMIAKNLRSIGIDVKTEFPDFSVWADRM...
20709493,1vr5A03,2o7iA02,55,DDDDDDDEHILMLMMMMMMMMMMMMMNOPACDDDDDDFKLMMMMMM...,HERNKTLYWGGALWSPPSNWNPFTPWNAVAGTIGLVYEPLFLYDPL...,ZZKLCCCDEHJACBDCFBFCFKLMMLGCEHJAOJLPCCFBDCDFKN...,SLPREDTVYIGGALWGPATTWNLYAPQSTWGTDQFMYLPAFQYDLG...,ZZFKLCCCDEHKLGCJDFBDCFKBMBDCEHJLMMMMCCFBDCDFKL...,355,410,369,424,FKLTIECPYGWTDWMVSIQSIAEDLVKVGINVEPKYPDYSKYADDL...,GPYTISVPYGWTDWMMMCEMIAKNLRSIGIDVKTEFPDFSVWADRM...


In [97]:
df.loc[15583196].x_pdb_seq

'HERNKTLYWGGALWSPPSNWNPFTPWNAVAGTIGLVYEPLFLYDPLNDKFEPWLAEKGEWVSNNEYVLTLRKGLRWQDGVPLTADDVVFTFEIAKKYTGISYSPVWNWLGRIERVDERTLKFVFSDPRYQEWKQMLINTPIVPKHIWENKTEEEVLQAANENPVGSGPYYVESWADDRCVFKKNGNWWGIRELGYDPKPERIVELRVLSNNVAVGMLMKGELDWSNFFLPGVPVLKKAYGIVTWYENAPYMLPANTAGIYINVNKYPLSIPEFRRAMAYAINPEKIVTRAYENMVTAANPAGILPLPGYMKYYPKEVVDKYGFKYDPEMAKKILDELGFKDVNKDGFREDPNGKPFKLTIECPYGWTDWMVSIQSIAEDLVKVGINVEPKYPDYSKYADDLYGGKFDLILNNFTTGVSATIWSYFNGVFYPDAVESEYSYSGNFGKYANPEVETLLDELNRSNDDAKIKEVVAKLSEILLKDLPFIPLWYNGAWFQASEAVWTNWPTEKNPYAVPIGWNGWWQLTGIKTLFGIEAK'

In [98]:
df.loc[15583196].y_pdb_seq

'SLPREDTVYIGGALWGPATTWNLYAPQSTWGTDQFMYLPAFQYDLGRDAWIPVIAERYEFVDDKTLRIYIRPEARWSDGVPITADDFVYALELTKELGIGPGGGWDTYIEYVKAVDTKVVEFKAKEENLNYFQFLSYSLGAQPMPKHVYERIRAQMNIKDWINDKPEEQVVSGPYKLYYYDPNIVVYQRVDDWWGKDIFGLPRPKYLAHVIYKDNPSASLAFERGDIDWNGLFIPSVWELWEKKGLPVGTWYKKEPYFIPDGVGFVYVNNTKPGLSDPAVRKAIAYAIPYNEMLKKAYFGYGSQAHPSMVIDLFEPYKQYIDYELAKKTFGTEDGRIPFDLDMANKILDEAGYKKGPDGVRVGPDGTKLGPYTISVPYGWTDWMMMCEMIAKNLRSIGIDVKTEFPDFSVWADRMTKGTFDLIISWSVGPSFDHPFNIYRFVLDKRLSKPVGEVTWAGDWERYDNDEVVELLDKAVSTLDPEVRKQAYFRIQQIIYRDMPSIPAFYTAHWYEYSTKYWINWPSEDNPAWFRPSPWHADAWPTLFIISKKSDPQPVPSWLGTVDEGGIEIPTAKIFEDLQKAT'

In [99]:
df.loc[15583196].x_pb_seq

'ZZKLCCCDEHJACBDCFBFCFKLMMLGCEHJAOJLPCCFBDCDFKNOPADDFKBMBDCDFBFKBCCDDDDEHIACFKOPACDFKLMMMMMMMMMMMMMBMKLMMMMMPMBDCDFBLMBCCDDDFBDCFKLMMMMMMMMCCDDFKLMMMPCFKLMMMPNOGAIACFBFBDCFBDCFKBCCDDDDFKBMKLMMNOPACFBLBDCDDDDFBFKLMMMMMMNOPAFBDCBDCEHKLMMMMMNOPADFBLBDEHIACFBDCDDDDDFKBCFKLPMKLMMMMMMMPCFKLMMMMMNNOPACCDDFKLPACBFKLMMMOCFKLMMMMMCCDDFKLMMMMMMMNOPACFKONOPACDFKOPACDDDDDDDEHILMLMMMMMMMMMMMMMNOPACDDDDDDFKLMMMMMMNOPAFBDCDFKLMGCCFBFKLMMMMMXXXXLMMPMBACKBJKLOPACFKLMMMMMMMMMPCBFKLMMMMMMMMMMMMMMNOMACCFBDCDDDDDDDFKLBGHIACFKLCCBCCDFBEHJLMNOMMMMMMMGCCZZ'

In [100]:
df.loc[15583196].y_pb_seq

'ZZFKLCCCDEHKLGCJDFBDCFKBMBDCEHJLMMMMCCFBDCDFKLOPADDFKBMBDCDFBLMBCCDDDDFKLCCFKOPACDFKLMMMMMMMMMMNOPAEHIHKLMMMMBACDFBFKLCCDDDDFKLGCFKLMMMMMMMPCCDDFKLMMMMMMMMCFKLPNOLAFKLPCCFBFBDCFBDCFKBCCDDDDFKLFKLMMNOPAFBLBDCDDDDFBFKLMMMMMMNOPAFBDCBDCEHKLLMMMMNOPACDDFBLBDEHIACFBDCDDDDDFKBGHJLPMKLMMMMMMMPCFKLMMMMMNNOPACCDDFKLPAFKBMKLMMMPCFKLMMMMNOPANOPACDDFKLMMMMMMMNOPACDFKOPACDFKLCCEBDDDDDDDEHILMLMMMMMMMMMMMMMNOPACDDDDDDFKLMMMMMMNOPAFBDCDDFBDDDFKBCKLMMMMMMMMKLMCCEHIACFKBOKLOPACFKLMMMMMMMMMPFCLKLMMMMMMMMMMMMMMNOMBACFBDCDDDDDDDFKLBGHIACFKLMMBFCLBFKBMKLMMMMMMPCCFKLCCDDDFKGCCFKNOPABDCFKLMMMMMMMMZZ'

In [102]:
df.loc[15583196].hit

'DDDDDDDEHILMLMMMMMMMMMMMMMNOPACDDDDDDFKLMMMMMMNOPAFBDCD'

In [105]:
I = 15583196
df.loc[I].x_pdb_seq[df.loc[I].x_hit_start:df.loc[I].x_hit_end]  

'FKLTIECPYGWTDWMVSIQSIAEDLVKVGINVEPKYPDYSKYADDLYGGKFDLIL'

In [106]:
I = 15583196
df.loc[I].y_pdb_seq[df.loc[I].y_hit_start:df.loc[I].y_hit_end]  

'GPYTISVPYGWTDWMMMCEMIAKNLRSIGIDVKTEFPDFSVWADRMTKGTFDLII'

In [106]:
df = pd.merge(df, pdb_intervals, how="inner", left_on="x", right_on="pdb").drop(columns="pdb").rename(columns={"range":"x_range"})
df = pd.merge(df, pdb_intervals, how="inner", left_on="y", right_on="pdb").drop(columns="pdb").rename(columns={"range":"y_range"})

In [108]:
def isIdentity(v):
    x = v[0]
    y = v[1]
    x_hit_start = v[2]
    x_hit_end = v[3]
    y_hit_start =  v[4]
    y_hit_end = v[5]
    x_frag = x[x_hit_start-1:x_hit_end]
    y_frag = y[y_hit_start-1:y_hit_end]
    return x_frag != y_frag
    
isIdentity = df[["x_pdb_seq", "y_pdb_seq", "x_hit_start", "x_hit_end","y_hit_start", "y_hit_end"]].apply(lambda x: isIdentity(x.values),axis=1)
df = df[isIdentity]
df.shape

(1000, 14)

In [109]:
df.head()

,x,y,hit_length,hit,x_pdb_seq,x_pb_seq,y_pdb_seq,y_pb_seq,x_hit_start,x_hit_end,y_hit_start,y_hit_end,x_range,y_range
0,12asA00,132lA00,14,DFKLMMMMMMMNOP,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...,ZZLMMMMMMMMMMMMMMMMMMMNOPACDDDDFBDCEHILPACEKJO...,XVFGRCELAAAMXRHGLDNYRGYSLGNWVCAAXFESNFNTQATNRN...,ZZDFKLMMMMMMMNOPGOPGHIAFKLMMMMMMMMNOOPAKBCCDDF...,253,266,3,16,4-330:A,2-129:A
1,12asA00,132lA00,14,DFKLMMMMMMMNOP,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...,ZZLMMMMMMMMMMMMMMMMMMMNOPACDDDDFBDCEHILPACEKJO...,XVFGRCELAAAMXRHGLDNYRGYSLGNWVCAAXFESNFNTQATNRN...,ZZDFKLMMMMMMMNOPGOPGHIAFKLMMMMMMMMNOOPAKBCCDDF...,253,266,3,16,4-330:A,2-129:A
2,12asA00,153lA00,26,MMMMMMMMMMMMMMMMMMMMMMNOPA,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...,ZZLMMMMMMMMMMMMMMMMMMMNOPACDDDDFBDCEHILPACEKJO...,RTDCYGNVNRIDTTGASCKTAKPEGLSYCGVSASKKIAERDLQAMD...,ZZMNOCFKLPCCEBJAFKLMMMNOPABECJLLMMMMMMMMMMMMMM...,129,154,36,61,4-330:A,1-185:A
3,12asA00,153lA00,26,MMMMMMMMMMMMMMMMMMMMMMNOPA,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...,ZZLMMMMMMMMMMMMMMMMMMMNOPACDDDDFBDCEHILPACEKJO...,RTDCYGNVNRIDTTGASCKTAKPEGLSYCGVSASKKIAERDLQAMD...,ZZMNOCFKLPCCEBJAFKLMMMNOPABECJLLMMMMMMMMMMMMMM...,129,154,36,61,4-330:A,1-185:A
4,12asA00,155cA00,13,FKLMMMMMMMMMM,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...,ZZLMMMMMMMMMMMMMMMMMMMNOPACDDDDFBDCEHILPACEKJO...,XNEGDAAKGEKEFNKCKACHMIQAPDGTDIKGGKTGPNLYGVVGRK...,ZZXPFKMMMMMMMXXXXXXXXXDCKBCEDJDDFKBACFEHJAOIAG...,126,138,71,83,4-330:A,1-134:A


In [119]:
df.loc[:, "x_hit_range"] = df[["x_range", "x_hit_start", "x_hit_end"]]\
                             .apply(lambda x: merge_range(scale_hit(parse_interval(x.x_range), x.x_hit_start, x.x_hit_end)), \
                                    axis=1)

df.loc[:, "y_hit_range"] = df[["y_range", "y_hit_start", "y_hit_end"]]\
                            .apply(lambda x: merge_range(scale_hit(parse_interval(x.y_range), x.y_hit_start, x.y_hit_end)), \
                                   axis=1)

In [125]:
df.rename(columns = {"x" : "x_pdb_id", "y" : "y_pdb_id"})\
[["x_pdb_id", "y_pdb_id", "x_hit_range", "y_hit_range", "hit", "hit_length"]]#.to_csv("hits_pb_alphabet.csv", index=False)
# df.head()

,x_pdb_id,y_pdb_id,x_hit_range,y_hit_range,hit,hit_length
0,12asA00,132lA00,256:269,4:17,DFKLMMMMMMMNOP,14
1,12asA00,132lA00,256:269,4:17,DFKLMMMMMMMNOP,14
2,12asA00,153lA00,132:157,36:61,MMMMMMMMMMMMMMMMMMMMMMNOPA,26
3,12asA00,153lA00,132:157,36:61,MMMMMMMMMMMMMMMMMMMMMMNOPA,26
4,12asA00,155cA00,129:141,71:83,FKLMMMMMMMMMM,13
5,12asA00,155cA00,129:141,71:83,FKLMMMMMMMMMM,13
6,132lA00,155cA00,89:100,72:83,KLMMMMMMMMMM,12
7,153lA00,155cA00,109:121,71:83,FKLMMMMMMMMMM,13
8,12asA00,16pkA02,9:32,,MMMMMMMMMMMMMMMMMNOPACDD,24
9,12asA00,16pkA02,9:32,,MMMMMMMMMMMMMMMMMNOPACDD,24


In [138]:
df.loc[8]
print(len(df.loc[8].y_pdb_seq     ) )
# print(len(df.loc[8].y_pb_seq      ))

415


In [120]:
# x_range = df.x_hit_range.apply(lambda x: x.split(":"))
# y_range = df.y_hit_range.apply(lambda x: x.split(":"))
# df["x_start"] = [x[0] for x in x_range]
# df["x_end"] = [x[1] for x in x_range]
# df["y_start"] = [x[0] for x in y_range]
# df["y_end"] = [x[1] for x in y_range]

# df.rename(columns = {"x" : "x_pdb_id", "y" : "y_pdb_id"})[["x_pdb_id", "y_pdb_id", "x_start", "x_end", "y_start", "y_end", "hit", "hit_length"]].to_csv("hits_chem_alphabet.csv", index=False)

IndexError: list index out of range

In [122]:
df.head()

,x,y,hit_length,hit,x_pdb_seq,x_pb_seq,y_pdb_seq,y_pb_seq,x_hit_start,x_hit_end,y_hit_start,y_hit_end,x_range,y_range,x_hit_range,y_hit_range,x_start
0,12asA00,132lA00,14,DFKLMMMMMMMNOP,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...,ZZLMMMMMMMMMMMMMMMMMMMNOPACDDDDFBDCEHILPACEKJO...,XVFGRCELAAAMXRHGLDNYRGYSLGNWVCAAXFESNFNTQATNRN...,ZZDFKLMMMMMMMNOPGOPGHIAFKLMMMMMMMMNOOPAKBCCDDF...,253,266,3,16,4-330:A,2-129:A,256:269,4:17,256
1,12asA00,132lA00,14,DFKLMMMMMMMNOP,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...,ZZLMMMMMMMMMMMMMMMMMMMNOPACDDDDFBDCEHILPACEKJO...,XVFGRCELAAAMXRHGLDNYRGYSLGNWVCAAXFESNFNTQATNRN...,ZZDFKLMMMMMMMNOPGOPGHIAFKLMMMMMMMMNOOPAKBCCDDF...,253,266,3,16,4-330:A,2-129:A,256:269,4:17,256
2,12asA00,153lA00,26,MMMMMMMMMMMMMMMMMMMMMMNOPA,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...,ZZLMMMMMMMMMMMMMMMMMMMNOPACDDDDFBDCEHILPACEKJO...,RTDCYGNVNRIDTTGASCKTAKPEGLSYCGVSASKKIAERDLQAMD...,ZZMNOCFKLPCCEBJAFKLMMMNOPABECJLLMMMMMMMMMMMMMM...,129,154,36,61,4-330:A,1-185:A,132:157,36:61,132
3,12asA00,153lA00,26,MMMMMMMMMMMMMMMMMMMMMMNOPA,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...,ZZLMMMMMMMMMMMMMMMMMMMNOPACDDDDFBDCEHILPACEKJO...,RTDCYGNVNRIDTTGASCKTAKPEGLSYCGVSASKKIAERDLQAMD...,ZZMNOCFKLPCCEBJAFKLMMMNOPABECJLLMMMMMMMMMMMMMM...,129,154,36,61,4-330:A,1-185:A,132:157,36:61,132
4,12asA00,155cA00,13,FKLMMMMMMMMMM,AYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLS...,ZZLMMMMMMMMMMMMMMMMMMMNOPACDDDDFBDCEHILPACEKJO...,XNEGDAAKGEKEFNKCKACHMIQAPDGTDIKGGKTGPNLYGVVGRK...,ZZXPFKMMMMMMMXXXXXXXXXDCKBCEDJDDFKBACFEHJAOIAG...,126,138,71,83,4-330:A,1-134:A,129:141,71:83,129


In [117]:
cols = ["x_range", "x_hit_start", "x_hit_end"]
df.head()[cols].apply(lambda x: parse_interval(x.x_range), axis=1)



0    [[4, 4]]
1    [[4, 4]]
2    [[4, 4]]
3    [[4, 4]]
4    [[4, 4]]
dtype: object